In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
##Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from getpass import getpass
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import json

# core functions

In [6]:
def get_category_links(cat_element_list):
    ## get links categories link element
    ## find elements having tag name a and get the links
    links = []
    for cat_element in cat_element_list:
        name = cat_element.text
        link = cat_element.find_element('tag name','a').get_attribute('href')
        links.append((name,link))
    return links

def get_all_question_category_wise(cat_url):
    ## scrape all questions on a particular category pages( first 5 pages)
    print("inside get all question")
    all_questions = []

    for page in range(1,6):
        page_url = cat_url+'?pageno='+str(page)
        driver.get(page_url)
        sleep(4)
        try:
            question_element_list = driver.find_elements('xpath','//div[@class="sques_quiz"]')
        except NoSuchElementException:
            #if less then 5 pages
            break
        for q in question_element_list:
            all_questions.append(get_question_info(q))
    return all_questions

def get_question_info(question):
    ## for each questions scraped, collect the answer choices and the correct answer
    
    q_text = question.find_element('xpath','.//div[@class="wp_quiz_question testclass"]').text
    q_text = q_text.split('\n')[1]
    options = question.find_element('xpath','.//div[@class="wp_quiz_question_options"]').text.split('\n')
    question.find_element('xpath','.//a[@class="wp_basic_quiz_showans_btn"]').click()
    sleep(0.35)   ## wait for the to be button clicked properly
    correct_ans = question.find_element('xpath','.//div[@class="ques_answer"]').text.split()[2]
    try:
        notes = question.find_element('xpath','.//div[@class="answer_hint"]').text.split('\n')[1]
    except:
        notes = ''
    return (q_text,options,correct_ans,notes)

In [ ]:
categories = driver.find_elements('xpath','//div[@class="fivecol column"]')
gk_subjects = get_category_links(categories[:16])
gk_states = get_category_links(categories[16:])

# Collecting subject-wise GK

In [ ]:
##CONNECTING TO --a quiz website (www.example-quiz-site.com)
driver = webdriver.Chrome()
driver.get('www.example-quiz-site.com')

##GETTING GK_common_category   
## SUB_WISE_QUESTION_BANK SIZE:: 50*16 = 800

## a dictionary {category_name:questions_list}
SUB_WISE_QUESTION_BANK = {}
for category in gk_subjects:
    cat_name = category[0]
    cat_url = category[1]
    driver.get(cat_url)
    sleep(2)
    questions = get_all_question_category_wise(cat_url)
    SUB_WISE_QUESTION_BANK.update({cat_name:questions})

## save the collected subject-wise question in json file

In [ ]:
with open('general_category_question.json','w') as fp:
    json.dump(SUB_WISE_QUESTION_BANK,fp)

## Store questions to Database

In [ ]:
import mysql.connector
from getpass import getpass
import json

conn = mysql.connector.connect(user='dbuser',password=getpass(),host='127.0.0.1',database='dbname')
mycursor = conn.cursor()

# load data from our json file
# Insert the questions into the database
with open('general_category_question.json') as fp:
    question_raw = json.load(fp)
    
    for enum in enumerate(question_raw.keys()):
    category_id = enum[0]
    category_name = enum[1]
    query = 'INSERT INTO category (category_id,category_name) values(%s,%s)'
    val = (category_id,category_name)
    mycursor.execute(query,val)
    conn.commit()
    for question in question_raw[category_name]:
        q_id = question_global_id
        question_global_id += 1
        q_text = question[0]
        options = question[1]
        #unpack options
        opt1 = options[0]
        opt2 = options[1]
        opt3 = options[2]
        opt4 = options[3]
        
        ans = question[2]
        notes = question[3]
        #insert to db table question_bank
        query = 'INSERT INTO question_bank values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        val = (q_id,q_text,opt1,opt2,opt3,opt4,ans,notes,category_id)
        mycursor.execute(query,val)
        conn.commit()